In [ ]:
# =========================================
# Lending Club 風險預測模型訓練程式
# =========================================
# 作者：Louis Ko
# 作者：
# 功能說明：
#   - 用於訓練信貸風險預測模型，並輸出 .keras / .pkl / .json 檔案供 FastAPI 使用
#   - 可在 VSCode、Colab 或命令列執行
#   - 能自動忽略 Jupyter Notebook 的多餘參數 (--f=kernel.json)
# =========================================

import argparse, sys
from pathlib import Path
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib, json, os

# =========================================
# 第 1 步：解析指令參數（安全忽略 Notebook 的多餘參數）
# =========================================
def parse_args():
    parser = argparse.ArgumentParser(description="訓練 Lending Club 風險預測模型")
    parser.add_argument("--epochs", type=int, default=5, help="訓練的迭代次數（預設：5）")
    parser.add_argument("--batch-size", type=int, default=32, help="每批訓練資料量（預設：32）")
    parser.add_argument("--seed", type=int, default=42, help="隨機種子（預設：42）")
    parser.add_argument("--save-dir", type=str, default="model", help="模型輸出資料夾（預設：./model）")
    args, _ = parser.parse_known_args(sys.argv[1:])  # 安全忽略 --f=kernel.json 等多餘參數
    return args

# =========================================
# 第 2 步：自動判斷執行環境（Colab / Notebook / CLI）
# =========================================
def get_base_dir():
    try:
        base_dir = Path(__file__).resolve().parent.parent  # 一般腳本執行時的根目錄
    except NameError:
        base_dir = Path.cwd()  # Notebook 執行時的工作目錄
    if "/content" in str(base_dir):
        base_dir = Path("/content")
    return base_dir

# =========================================
# 第 3 步：建立模擬訓練資料（可改為真實 Lending Club 或內部資料）
# =========================================
def generate_mock_data(n=1000, seed=42):
    np.random.seed(seed)
    df = pd.DataFrame({
        'loan_amnt': np.random.randint(5000, 40000, n),    # 貸款金額
        'annual_inc': np.random.randint(30000, 200000, n), # 年收入
        'dti': np.random.uniform(5, 40, n),                # 債務比（Debt-to-Income ratio）
        'total_acc': np.random.randint(5, 40, n),          # 帳戶總數
        'revol_util': np.random.uniform(0, 100, n),        # 循環信貸使用率（%）
        'int_rate': np.random.uniform(5, 25, n),           # 利率（%）
        'loan_repaid': np.random.choice([0, 1], n, p=[0.25, 0.75])  # 是否繳清（0=違約，1=已繳清）
    })
    return df

# =========================================
# 第 4 步：模型訓練與輸出
# =========================================
def train_and_save(df, save_dir: Path, epochs: int, batch_size: int):
    # 建立輸出資料夾
    save_dir.mkdir(parents=True, exist_ok=True)

    # 分離特徵與標籤
    X = df.drop('loan_repaid', axis=1).values
    y = df['loan_repaid'].values

    # 特徵縮放
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    # 分割訓練集與測試集
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=101
    )

    # 建立簡易神經網路模型（可改為更進階架構）
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # 編譯模型
    model.compile(loss='binary_crossentropy', optimizer='adam')

    # 訓練模型
    model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test, y_test),
        verbose=1
    )

    # 儲存模型與前處理器
    model.save(save_dir / "LendingClub.keras")
    joblib.dump(scaler, save_dir / "scaler.pkl")

    # 儲存欄位名稱（API 端需依此順序組成輸入）
    columns = df.drop('loan_repaid', axis=1).columns.tolist()
    with open(save_dir / "columns.json", "w") as f:
        json.dump(columns, f, ensure_ascii=False, indent=2)

    print(f"\n模型與前處理器已儲存於: {save_dir.resolve()}")
    print(f"檔案清單:")
    for file in save_dir.iterdir():
        print(f" - {file.name}")

# =========================================
# 第 5 步：主程式入口點
# =========================================
if __name__ == "__main__":
    args = parse_args()
    base_dir = get_base_dir()
    model_dir = base_dir / args.save_dir

    print("=" * 60)
    print("Lending Club 風險模型訓練啟動")
    print(f"根目錄: {base_dir}")
    print(f"輸出位置: {model_dir}")
    print(f"訓練參數 -> Epochs: {args.epochs}, Batch Size: {args.batch_size}, Seed: {args.seed}")
    print("=" * 60)

    df = generate_mock_data(seed=args.seed)
    train_and_save(df, model_dir, args.epochs, args.batch_size)


Lending Club 風險模型訓練啟動
根目錄: /Users/louisko/fileproject/Wits-Hackathon/ml_module
輸出位置: /Users/louisko/fileproject/Wits-Hackathon/ml_module/model
訓練參數 -> Epochs: 5, Batch Size: 32, Seed: 42
Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.6397 - val_loss: 0.6310
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5917 - val_loss: 0.6149
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5729 - val_loss: 0.6157
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5670 - val_loss: 0.6199
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5658 - val_loss: 0.6194

模型與前處理器已儲存於: /Users/louisko/fileproject/Wits-Hackathon/ml_module/model
檔案清單:
 - scaler.pkl
 - columns.json
 - LendingClub.keras
